#### 상품페이지 가격정보(판매가, 할인가, 쿠폰적용가) 크롤링

In [1]:
#input
id_naver = '(아이디)'
pw_naver = '(비밀번호)'
id_gmarket = '(아이디)'
pw_gmarket = '(비밀번호)'
id_11st = '(아이디)'
pw_11st = '(비밀번호)'

url_naver = '(상품페이지 주소)','(상품페이지 주소)'
url_gmarket =  '(상품페이지 주소)','(상품페이지 주소)'
url_11st =  '(상품페이지 주소)','(상품페이지 주소)'

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import numpy as np 
import pandas as pd
import time
import random

mall = []
product = []
product_no = []
product_name = []
product_url = []
price_original = []
price_discount = []
price_final =  []
created_date = []

def getNaverPrice():
    try:
        time.sleep(1)
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        product_info = bs.find('tbody').find_all('td',{'ABROiEshTD'})
        product_no.append(product_info[0].get_text())
        product.append(product_info[4].get_text())
        product_name.append(bs.find('div',{'class':'_1eddO7u4UC'}).find('h3',{'_22kNQuEXmb _copyable'}).get_text())
        product_url.append(url)
        created_date.append(time.strftime("%Y-%m-%d"))
        mall.append('Naver')
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        benefit_button = bs.find('div',{'class':'_2a18RJADk5'}).find_all('button')
        price = bs.find('div',{'class':'_3my-5FC8OB'}).find_all('span',{'class':'_1LY7DqCnwR'})
        if len(price)== 1:
            price_original.append(price[0].get_text())
            price_discount.append(price[0].get_text())
        elif len(price)== 2:
            price_original.append(price[0].get_text())
            price_discount.append(price[1].get_text())
        if len(benefit_button) == 3:
            price_final.append(bs.find('div',{'class':'_1UNQIwX1sN'}).find('span',{'_1SXfO9k819'}).get_text())
        elif len(benefit_button) == 1:
            if len(price)== 1:
                price_final.append(price[0].get_text())
            elif len(price)== 2:
                price_final.append(price[1].get_text())
    except AttributeError:
        return None

def getGmarketCoupon():
    try:
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        driver.find_element_by_xpath('//*[@id="itemcase_basic"]/div[2]/a/span[3]').click()
        time.sleep(1)
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        coupon_list = bs.find('ul',{'class':'list__coupon'}).find_all('li')
        for n in range(1,len(coupon_list)+1):
            getcoupon ='body > div.js__vipci.section__iframe-vipcoupon.section__iframe-vipcoupon--active > div > div.box__coupon-content > div > ul.list__coupon > li:nth-child('+str(n)+') > div > div.box__coupon-state'
            getcoupon_closed = 'body > div.js__vipci.section__iframe-vipcoupon.section__iframe-vipcoupon--active > div > div.box__coupon-header > button'
            driver.find_element(By.CSS_SELECTOR,getcoupon).click()
            time.sleep(1)
        driver.find_element(By.CSS_SELECTOR,getcoupon).click()
        driver.find_element(By.CSS_SELECTOR,getcoupon_closed).click()
    except NoSuchElementException:
        return None 
    
def selectGmarketCoupon():
    try:
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        option_selector = '#noOptionSelectDiv > button'
        driver.find_element(By.CSS_SELECTOR,option_selector).click()
        driver.implicitly_wait(60)
        selectCoupon()
    except NoSuchElementException:
        selectCoupon()

def selectCoupon():
    try:
        coupon_selector = '#coreSelectedLi_0 > div.choose_calc > span.btns'
        driver.find_element(By.CSS_SELECTOR,coupon_selector).click()
        driver.implicitly_wait(60)
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        coupon_list = bs.find('div',{'class':'coupon'}).find_all('dl')
        gCoupon = coupon_list[1].find_all('span',{'class':'benefit'})
        sCoupon = coupon_list[2].find_all('span',{'class':'benefit'})
        if gCoupon[0].find('span').get_text().rstrip('원') >= gCoupon[1].find('span').get_text().rstrip('원'):
            driver.find_element_by_xpath('//*[@id="gCouponList"]/dd/ul/li[1]').click()
        elif gCoupon[0].find('span').get_text().rstrip('원') < gCoupon[1].find('span').get_text().rstrip('원'): 
            driver.find_element_by_xpath('//*[@id="gCouponList"]/dd/ul/li[2]').click()
        if len(sCoupon) > 1:
            driver.find_element_by_xpath('//*[@id="sCouponList"]/dd/ul/li[2]').click()
        elif len(sCoupon) == 1:
            pass
        driver.find_element_by_xpath('//*[@id="layer_couponbox"]/div[3]/div[4]').click()
    except NoSuchElementException:
        return None
    
def getGmarketPrice():
    try:
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        price_final.append(bs.find('div',{'class':'form_bottom'}).find('strong',{'class':'price'}).get_text().rstrip('원'))
        price = bs.find('p',{'class':'price'}).find_all('span')
        if len(price)== 3:
            price_original.append(price[1].find('strong',{'class':'price_real'}).get_text().rstrip('원'))
            price_discount.append(price[1].find('strong',{'class':'price_real'}).get_text().rstrip('원'))
        elif len(price)== 6:
            price_original.append(price[1].find('span',{'class':'text__price'}).get_text().rstrip('원'))
            price_discount.append(price[1].find('strong',{'class':'price_real'}).get_text().rstrip('원'))
        product_info = bs.find('div',{'class':'box__product-notice-list'}).find_all('td')
        product_no.append(product_info[0].get_text())
        product.append(product_info[8].get_text())
        product_name.append(bs.find('div',{'class':'item-topinfo_headline'}).find('h1',{'itemtit'}).get_text())
        product_url.append(url)
        created_date.append(time.strftime("%Y-%m-%d"))
        mall.append('Gmarket')
    except AttributeError:
        return None

def get11stCoupon():
    try:
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        driver.find_element(By.CSS_SELECTOR,'#couponDownButton').click()
        driver.implicitly_wait(30)
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        driver.find_element(By.CSS_SELECTOR,'#btn_down_all_cp').click()
        driver.implicitly_wait(30)
        driver.find_element(By.CSS_SELECTOR,'#couponDown > div.r-coupon-down__bottom > button.r-coupon-down__button-cancel').click()
    except NoSuchElementException:
        return None

def get11stPrice():
    try:
        time.sleep(5)
        bs = BeautifulSoup(driver.page_source, 'html.parser')
        price = bs.find('div',{'class':'c_prd_price c_product_price_style1'}).find_all('dl')
        price_coupon = bs.find('dl',{'class':'b_product_info_merit'}).find_all('dl')
        price_total = bs.find('div',{'class':'total_wrap c_product_buy_price'}).find_all('dl')
        if len(price)== 1:
            price_original.append(price[0].find('span').get_text())
            price_discount.append(price[0].find('span').get_text())
        elif len(price)>= 2:
            price_original.append(price[len(price)-1].find('span').get_text())
            price_discount.append(price[len(price)-2].find('span').get_text())
        if int(price_total[0].find('span').get_text().replace('\n','').replace(',','').strip(' ')) != 0:
            price_final.append(price_total[0].find('span').get_text())
        elif price_coupon[0].find('dt').get_text() == '배송 안내':
            price_final.append(price[0].find('span').get_text())
        else:
            price_final.append(price_coupon[0].find('span').get_text())
        product_no.append (bs.find('table',{'class':'prdc_detail_table'}).find_all('td')[1].get_text())
        product.append(bs.find('div',{'class' : 'l_product_side_info'}).find('h1').get_text().replace('\n','').strip(' '))
        product_name.append(bs.find('div',{'class' : 'l_product_side_info'}).find('h1').get_text().replace('\n','').strip(' '))
        product_url.append(url)
        created_date.append(time.strftime("%Y-%m-%d"))
        mall.append('11st')
    except AttributeError:
        return None

driver = webdriver.Chrome('chromedriver.exe')
driver.implicitly_wait(30)
driver.get('https://nid.naver.com/nidlogin.login?mode=form&url=https%3A%2F%2Fwww.naver.com')
driver.implicitly_wait(30)
for id in id_naver:
    time.sleep(0.05)
    driver.find_element_by_name('id').send_keys(id)
time.sleep(random.randrange(1,3))
for pw in pw_naver:
    time.sleep(0.05)
    driver.find_element_by_name('pw').send_keys(pw)
driver.find_element_by_xpath('//*[@id="log.login"]').click()
driver.implicitly_wait(30)
time.sleep(30)

driver.get('https://signinssl.gmarket.co.kr/login/login?url=https://www.gmarket.co.kr/')
driver.implicitly_wait(30)
for id in id_gmarket:
    time.sleep(0.05)
    driver.find_element_by_name('id').send_keys(id)
time.sleep(random.randrange(1,3))
for pw in pw_gmarket:
    time.sleep(0.05)
    driver.find_element_by_name('pwd').send_keys(pw)
driver.find_element_by_xpath('//*[@id="mem_login"]/div[1]/div[3]/button').click()
driver.implicitly_wait(30)

driver.get('https://11st.co.kr')
driver.implicitly_wait(30)
driver.get('https://login.11st.co.kr/auth/front/login.tmall?returnURL=https%3A%2F%2F11st.co.kr%2F')
driver.implicitly_wait(30)
for id in id_11st:
    time.sleep(0.05)
    driver.find_element_by_name('loginName').send_keys(id)
time.sleep(random.randrange(1,3))
for pw in pw_11st:
    time.sleep(0.05)
    driver.find_element_by_name('passWord').send_keys(pw)
driver.find_element_by_xpath('//*[@id="loginbutton"]').click()
driver.implicitly_wait(30)

In [4]:
for url in coupon_11st :
    driver.get(url)
    driver.implicitly_wait(60)
    get11stCoupon()

for url in coupon_gmarket :
    driver.get(url)
    driver.implicitly_wait(60)
    getGmarketCoupon()

In [12]:
for url in url_naver:
    driver.get(url)
    driver.implicitly_wait(60)
    getNaverPrice()

for url in url_gmarket:
    driver.get(url)
    driver.implicitly_wait(60)
    selectGmarketCoupon()
    getGmarketPrice()

for url in url_11st:
    driver.get(url)
    driver.implicitly_wait(60)
    get11stPrice()

In [139]:
result = {"mall" : mall, "product_no" : product_no, "product" : product, "product_name" : product_name,#"product_url" : product_url,
                  "price_original" : price_original, "price_discount" : price_discount,"price_final" : price_final,
                  "created_date" : created_date}
df = pd.DataFrame(result)
df

,mall,product_no,product,product_name,price_original,price_discount,price_final,created_date
0,Naver,0000000000,A0000/RD,노출상품명 1,"744,400","399,000","359,100",2023-01-12
1,Naver,1111111111,A0000/BL,노출상품명 2,"613,000","379,000","341,100",2023-01-12
2,Gmarket,2222222222,A0000/PP,노출상품명 3,"475,000","329,000","249,000",2023-01-12
3,Gmarket,3333333333,A0000/BK,노출상품명 4,"359,000","329,000","329,000",2023-01-12
4,11st,4444444444,A0000/YE,노출상품명 5,"342,780","322,220","290,000",2023-01-12
5,11st,5555555555,C0000/WT,노출상품명6,"367,000","344,980","310,490",2023-01-12
